In [ ]:
import os
import leafmap
import geopandas as gpd
import pandas as pd

In [ ]:
in_dir = '/media/hdd/Team-Drives/Buildings/MS_USBuildingHeight/'
out_dir = '/media/hdd/Team-Drives/Buildings/Statistics/'

In [ ]:
files = leafmap.find_files(in_dir, ext='*.zip')
files.sort()

In [ ]:
for file in files:
    basename = os.path.basename(file)
    out_csv = os.path.join(out_dir, basename.replace('.zip', '.csv'))
    state = basename.split('.')[0]
    # if state not in ['AK', 'DC']:
    #     continue
    print(f'Processing {state} ...')
    gdf = gpd.read_file(file)

    df = gdf.describe().transpose()
    df['median'] = df['50%']
    df['state'] = state
    df.drop(['25%', '50%', '75%'], axis=1, inplace=True)
    df = df[['state', 'count', 'mean', 'std', 'min', 'median', 'max']]
    df = df.round(2)
    df['count'] = df['count'].astype(int)

    for item in ['AREA', 'HEIGHT', 'SQMETERS']:
        out_csv = os.path.join(out_dir, item + '.csv')
        if not os.path.exists(out_csv):
            stats = df.loc[[item]]
            stats.to_csv(out_csv, index=False)
        else:
            stats = pd.read_csv(out_csv)
            new_row = df.loc[[item]].reset_index().to_dict('records')[0]
            new_row.pop('index')
            stats = stats.append(new_row, ignore_index=True)
            stats.to_csv(out_csv, index=False)

In [ ]:
files = leafmap.find_files(out_dir, ext='*.csv')
files.sort()
files

In [ ]:
for file in files:
    df = pd.read_csv(file)
    df['total'] = df['count'] * df['mean']
    df['total'] = df['total'].astype(int)
    df.to_csv(file, index=False)

In [ ]:
in_dir = '/media/hdd/Team-Drives/Buildings/Validation/'
out_dir = '/media/hdd/Team-Drives/Buildings/Statistics/'

In [ ]:
files = leafmap.find_files(in_dir, ext='*.shp', recursive=False)
files.sort()

In [ ]:
files

In [ ]:
for index, file in enumerate(files):
    if index < 0:
        continue
    basename = os.path.basename(file)
    print(f'Processing {basename} ...')

    gdf = gpd.read_file(file)
    gdf['difference'] = gdf['HEIGHT'] - gdf['est_height']
    df = gdf.describe().transpose()
    
    df['median'] = df['50%']
    df['county'] = basename.replace('.shp', '')
    df['index'] = df.index
    df.drop(['25%', '50%', '75%'], axis=1, inplace=True)
    df = df[['index', 'county', 'count', 'mean', 'std', 'min', 'median', 'max']]
    df = df.round(2)
    df['count'] = df['count'].astype(int)

    out_csv = os.path.join(out_dir, 'VALIDATION.csv')
    if not os.path.exists(out_csv):
        df.to_csv(out_csv, index=False)
    else:
        stats = pd.read_csv(out_csv)
        stats = stats.append(df, ignore_index=True)
        stats.sort_values(by=['index'], inplace=True)
        stats.to_csv(out_csv, index=False)
